First we are going to parse the annotation file and see what we can learn...

In [1]:
from rdflib import Graph, URIRef
from rdflib.namespace import DCTERMS

# use this URI to identify delayed variables - not the perfect URI, but will do for now
delay_variable_uri = URIRef('https://github.com/nickerso/libcellml-python-utils/properties.rst#delay-variable')
variable_to_delay_uri = URIRef('https://github.com/nickerso/libcellml-python-utils/properties.rst#variable-to-delay')

# a "readout" variable that we maybe want to connect to something external?
timecourse_readout_uri = URIRef('http://identifiers.org/mamo/MAMO_0000031')

annotation_file = 'models/sine_approximations-updated-ids--annotations.ttl'
g = Graph().parse(annotation_file)

# find all delayed variables
variables_to_delay = []
for d in g.subjects(DCTERMS.type, variable_to_delay_uri):
    # we only expect one delay variable for each variable to delay
    dv = g.value(d, delay_variable_uri)
    variables_to_delay.append([str(d), str(dv)])
    
print(variables_to_delay)

# find all timecourse readouts
readout_variables = []
for d in g.subjects(DCTERMS.type, timecourse_readout_uri):
    readout_variables.append(str(d))
    
print(readout_variables)

[['file:///C:/Users/dnic019/Documents/projects/libCellML/python-utils/libcellml-python-utils/models/sine_approximations-updated-ids.xml#b4da83', 'file:///C:/Users/dnic019/Documents/projects/libCellML/python-utils/libcellml-python-utils/models/sine_approximations-updated-ids.xml#b4da82']]
['file:///C:/Users/dnic019/Documents/projects/libCellML/python-utils/libcellml-python-utils/models/sine_approximations-updated-ids.xml#b4da83', 'file:///C:/Users/dnic019/Documents/projects/libCellML/python-utils/libcellml-python-utils/models/sine_approximations-updated-ids.xml#b4da84', 'file:///C:/Users/dnic019/Documents/projects/libCellML/python-utils/libcellml-python-utils/models/sine_approximations-updated-ids.xml#b4da86', 'file:///C:/Users/dnic019/Documents/projects/libCellML/python-utils/libcellml-python-utils/models/sine_approximations-updated-ids.xml#b4da82']


We're going to use the "model" file from the first variable to delay and only continue if all annotations use the same model...

In [2]:
from urllib.parse import urlparse

model_uri = variables_to_delay[0][0]
model_url = urlparse(model_uri)
model_file = model_url.path

delayed_ids = []
for v, dv in variables_to_delay:
    url = urlparse(v)
    if url.path != model_file:
        print("found an unexpected model file for variable to delay?!")
        exit
    dv_url = urlparse(dv)
    if dv_url.path != model_file:
        print("found an unexpected model file for delay variable?!")
        exit
    delayed_ids.append([url.fragment, dv_url.fragment])
    
readout_ids = []
for v in readout_variables:
    url = urlparse(v)
    if url.path != model_file:
        print("found an unexpected model file for readout variable?!")
        exit
    readout_ids.append(url.fragment)

Now we have the model file and the IDs for the variables in that model that we want to do stuff with. So we can parse the model and see if we can find the variables.

In [3]:
import cellml

# on windows getting a leading '/' in the filename which libCellML doesn't like...
fixed_model_file = model_file[1:]

# parse the model in non-strict mode to allow non CellML 2.0 models
model = cellml.parse_model(fixed_model_file, False)

# and make an annotator for this model
from libcellml import Annotator
annotator = Annotator()
annotator.setModel(model)

# map our IDs to the actual variables
annotated_variables = []
for i, dv_i in delayed_ids:
    # get the variable (will fail if id doesn't correspond to a variable in the model)
    v = annotator.variable(i)
    if v == None:
        print('Unable to find a variable to delay with the id {} in the given model...'.format(i))
        exit
    dv = annotator.variable(dv_i)
    if dv == None:
        print('Unable to find a delay variable with the id {} in the given model...'.format(dv_i))
        exit
    annotated_variables.append([[v, dv], delay_variable_uri])
    
for i in readout_ids:
    # get the variable (will fail if id doesn't correspond to a variable in the model)
    v = annotator.variable(i)
    if v == None:
        print('Unable to find a readout variable with the id {} in the given model...'.format(i))
        exit
    annotated_variables.append([v, timecourse_readout_uri])

# TODO:
Need to work out how to map the annotations through to the variables in the generated code....

Generate C code for the model.

In [21]:
import os 
model_dir = os.path.dirname(fixed_model_file)

# resolve imports, in non-strict mode
importer = cellml.resolve_imports(model, model_dir, False)
# need a flattened model for analysing
flat_model = cellml.flatten_model(model, importer)

from libcellml import Analyser, AnalyserModel, AnalyserExternalVariable, Generator, GeneratorProfile        

# analyse the model
a = Analyser()

# set the delayed variables as external
external_variable_info = []
for vv, t in annotated_variables:
    if t == delay_variable_uri:
        v = vv[0]
        dv = vv[1]
        flat_variable = flat_model.component(v.parent().name()).variable(v.name())
        flat_delay_variable = flat_model.component(dv.parent().name()).variable(dv.name())
        aev = AnalyserExternalVariable(flat_variable)
        aev.addDependency(flat_delay_variable)
        #
        # TODO: really need to work out how to handle other dependencies here to make sure 
        #       all required variables are up to date...
        #
        a.addExternalVariable(aev)
        # keep track of external variable information for use in generating code
        external_variable_info.append({
            'variable': flat_variable,
            'delay_variable': flat_delay_variable,
            'analyser_variable': aev
        })

a.analyseModel(flat_model)
analysed_model = a.model()
print(analysed_model.type())

# get the information for the variables to delay
for ext_variable in external_variable_info:
    ev = ext_variable['variable']
    avs = analysed_model.variables()
    for av in avs:
        v = av.variable()
        if analysed_model.areEquivalentVariables(v, ext_variable['variable']):
            ext_variable['index'] = av.index()

# generate code from the analysed model
g = Generator()
# using the C profile to generate C code
profile = GeneratorProfile(GeneratorProfile.Profile.C)
g.setProfile(profile)
g.setModel(analysed_model)

# print the C header code
print('header code:')
print(g.interfaceCode())

# print the C implementation code
print('implementation code:')
print(g.implementationCode())

# and generate a function to compute external variables
computeEV = f"""
#include <stdlib.h>
double computeExternalVariable(double voi, double *states, double *variables, size_t index)
{{
  
"""
for ext_variable in external_variable_info:
    index = ext_variable['index']
    v = ext_variable['variable']
    computeEV += f'  if (index == {index}) {{\n'
    computeEV += f'    return delay({v.name()}, voi - 3.0);\n'
    computeEV += f'  }}\n'

computeEV += f"""
  return 0.0;
}}
"""
print('compute external variables implementation:')
print(computeEV)

no unresolved imports.
2
header code:
/* The content of this file was generated using the C profile of libCellML 0.3.104. */

#pragma once

#include <stddef.h>

extern const char VERSION[];
extern const char LIBCELLML_VERSION[];

extern const size_t STATE_COUNT;
extern const size_t VARIABLE_COUNT;

typedef enum {
    VARIABLE_OF_INTEGRATION,
    STATE,
    CONSTANT,
    COMPUTED_CONSTANT,
    ALGEBRAIC,
    EXTERNAL
} VariableType;

typedef struct {
    char name[27];
    char units[14];
    char component[21];
    VariableType type;
} VariableInfo;

extern const VariableInfo VOI_INFO;
extern const VariableInfo STATE_INFO[];
extern const VariableInfo VARIABLE_INFO[];

double * createStatesArray();
double * createVariablesArray();
void deleteArray(double *array);

typedef double (* ExternalVariable)(double voi, double *states, double *variables, size_t index);

void initialiseVariables(double voi, double *states, double *variables, ExternalVariable externalVariable);
void computeCompute